# Hello World, "Hamilton Heater Shaker"!

The Hamilton Heater Shaker is a `HeaterShaker` machine that enables:
- heating, 
- locking & unlocking, and
- (orbital) shaking

...of plates (active cooling is not possible with this machine).

- [manufacturer_link](https://www.hamiltoncompany.com/automated-liquid-handling/small-devices/hamilton-heater-shaker?srsltid=AfmBOooBVzRaBrPj4UYumvbcbECIxj4lk_0jpJDjMrLksnFJPOgNURm6)
- Temperature control = RT+5°C to 105°C (all variants)
- Variants:
    - Cat. no.: 199027  
        - shaking orbit = 1.5 mm  
        - shaking speed = 100 - 1800 rpm
    - Cat. no.: 199033  
        - shaking orbit = 2.0 mm  
        - shaking speed = 100 - 2500 rpm
    - Cat. no.: 199034  
        - shaking orbit = 3.0 mm  
        - shaking speed = 100 - 2400 rpm  
        - max. loading = 500 grams  

- Footprint: size_x = 146.2, size_y = 103.8, size_z=74.11

---
## Setup Instructions (Physical)

A Hamilton Heater Shaker (hhs) can be used through two different **control interfaces**:
- a **control box**, called the `HamiltonHeaterShakerBox`: this supports connection of up to **8 heater shakers** per control box, OR
- **directly plugging** the hhs **into a Hamilton STAR liquid handler**: STAR liquid handlers have 2 RS232 ports on their left side, and can therefore support up to **2 heater shakers** simultaneously.

When using the **heater shaker box control interface** a USB-B cable is plugged into one of the heater shakers and connected to the host computer.
This heater shaker is connected via a serial port to the control box. Other heater shakers are also plugged into the control box using serial cables, but not plugged into the computer.
The first heater shakers serves as a gateway.

When using the **Hamilton STAR interface**, the Heater Shaker is connected via a serial interface:
- Connection: STAR (RS232) ⇄ Host computer (USB-A)

The heater shaker is then controlled through the STAR Liquid Handler.

---
## Setup Instructions (Programmatic)

In either case, `HamiltonHeaterShakerBackend` will be the backend and `HeaterShaker` will be the frontend.
Depending on the interface you use, pass a different argument to `HamiltonHeaterShakerBackend`.

**hs_box_control**:
As multiple heater shakers can be controlled through one USB connection to the computer (a cable to HHS 1 when using the control box), the `index` of a specific heater shaker needs to be specified.
Note that this also requires turing a DIP switch on the bottom of the HHS module.

**star_control**:
Each heater shaker is connected via a separate cable to the STAR liquid handler.
The back RS232 port corresponds to `index=1` and the front RS232 port corresponds to `index=2`.

In [1]:
interface_choice = "star_control" # hs_box_control VS star_control

In [ ]:
if interface_choice == "hs_box_control":
    
    # Setting up a backend with the HamiltonHeaterShakerBox
    from pylabrobot.heating_shaking import HamiltonHeaterShakerBackend, HamiltonHeaterShakerBox
    
    control_interface = hhs_box = HamiltonHeaterShakerBox()

elif interface_choice == "star_control":
    
    # Alternative: setting up a backend with a STAR
    from pylabrobot.liquid_handling import LiquidHandler, STARBackend
    from pylabrobot.resources import STARDeck
    from pylabrobot.heating_shaking import HamiltonHeaterShakerBackend
    
    control_interface = star_backend = STARBackend()

    # Control via a STAR requires instantiation of the STAR liquid handler
    lh = LiquidHandler(backend=star_backend, deck=STARDeck())

else:
    raise ValueError(f"Interface choice invalid: {interface_choice}")

backend = HamiltonHeaterShakerBackend(
    index=1,
    interface=control_interface
)

In [3]:
from pylabrobot.heating_shaking import HeaterShaker
from pylabrobot.resources.coordinate import Coordinate

hs = HeaterShaker(
  name="Hamilton HeaterShaker",
  backend=backend,
  size_x=146.2,
  size_y=103.8,
  size_z=74.11,
  child_location=Coordinate(x=9.66, y=9.22, z=74.11),
)

Note that you will need to call `hhs_box.setup()` before calling `HeaterShaker.setup()`.
When using a `STAR`, just use `star.setup()` or, more likely, `lh.setup()`.
This is opening the USB connection to the device you are using as an interface.

Note that setup should only be called ONCE:
- when using a STAR as a liquid handler, just call `lh.setup()`.
(Do not call it again when using the heater shaker.)
- when using multiple heater shakers with the control box, call `.setup()` once for the control box, and then call `HeaterShaker.setup()` for each heater shaker.
(Do not call `setup` again for the control box.)

In [4]:
if interface_choice == "hs_box_control":
    
    # When using the HamiltonHeaterShakerBox, you need to call setup() on the box
    await hhs_box.setup()

elif interface_choice == "star_control":

    # Alternative: when using the STAR, you need to call setup() on lh
    await lh.setup()

After calling `setup` on your interface, call `HeaterShaker.setup()` for each heater shaker machine.
This will initialize the `HeaterShaker` machine itself.

In [6]:
await hs.setup()

### Assigning a Hamilton Heater Shaker to the deck

Before you can use the Hamilton Heater Shaker in combination with a Hamilton STAR liquid handler, you need to assign it to the deck. This is needed when, for example, you want to use the iSWAP or CoRe grippers to move a plate to or from the heater shaker. This is also required to get the heater shaker to show up in the Visualizer.

Here's one example of assigning a Hamilton Heater Shaker to the deck using a `MFX_CAR_P3_SHAKER`. Note that you can use any carrier, or even directly place heater shakers on the deck if you like. See the [Hamilton STAR resources page](/resources/library/hamilton) for carriers.

In [ ]:
shaker_carrier = MFX_CAR_P3_SHAKER(name="shaker_carrier", modules={0: hs2, 1: hs1, 2: hs0})
lh.deck.assign_child_resource(shaker_carrier, rails=5)

---
## Usage

### Heating Control

In [7]:
await hs.get_temperature() # Temperature of sensor in the middle of the heater shaker in C

25.6

The HHS also supports reading the temperature at the edge of the heater shaker:

In [8]:
await hs.backend.get_edge_temperature()

25.7

In [6]:
await hs.set_temperature(37) # Temperature in degrees C

'T1TAid0004er00'

In [7]:
await hs.wait_for_temperature()  # Wait for the temperature to stabilize

### Shaking Control

In [9]:
await hs.lock_plate()

In [10]:
await hs.shake(
  speed=100, # rpm
  direction=0, # seconds
  acceleration=1000, # rpm/sec
) 

In [12]:
await hs.stop_shaking()

In [11]:
await hs.unlock_plate()

### Closing Connection to Machine

In [9]:
await hs.stop()

---
## Using Multiple Hamilton Heater Shakers

### 1x hs_box - Multiple HHS

When using multiple heater shakers, you can use the `HamiltonHeaterShakerBackend` class to control them. This class will automatically handle the communication with the control box and the individual heater shakers.

As above, initialize the `HamiltonHeaterShakerBox` class. Then, initialize as many `HamiltonHeaterShakerBackend` classes as you want to control, specifying the index for each. Note that each `HamiltonHeaterShakerBackend` gets the same instance of the `HamiltonHeaterShakerBox`: this is because there is a single USB connection, managed by that instance.

In [ ]:
control_interface = hhs_box = HamiltonHeaterShakerBox()

# HS1
backend1 = HamiltonHeaterShakerBackend(index=1, interface=control_interface)
hs1 = HeaterShaker(
  name="Hamilton HeaterShaker",
  backend=backend1,
  size_x=146.2,
  size_y=103.8,
  size_z=74.11,
  child_location=Coordinate(x=9.66, y=9.22, z=74.11),
)

# HS2
backend2 = HamiltonHeaterShakerBackend(index=2, interface=control_interface)
hs2 = HeaterShaker(
  name="Hamilton HeaterShaker",
  backend=backend2,
  size_x=146.2,
  size_y=103.8,
  size_z=74.11,
  child_location=Coordinate(x=9.66, y=9.22, z=74.11),
)

For setup, call `setup` on the `HamiltonHeaterShakerBox` instance. This will setup the USB connection to the control box. Then, call `setup` on each `HamiltonHeaterShakerBackend` instance. This will setup the individual heater shakers.

In [ ]:
await hhs_box.setup()

for hs in [hs1, hs2]:
  await hs.setup()

### 1x STAR - 2x hhs


In [ ]:
control_interface = star_backend = STARBackend()

# Control via a STAR requires instantiation of the STAR liquid handler
lh = LiquidHandler(backend=star_backend, deck=STARDeck())

# HS1
backend1 = HamiltonHeaterShakerBackend(index=1, interface=control_interface)

hs1 = HeaterShaker(
  name="Hamilton HeaterShaker",
  backend=backend1,
  size_x=146.2,
  size_y=103.8,
  size_z=74.11,
  child_location=Coordinate(x=9.66, y=9.22, z=74.11),
)

# HS2
backend2 = HamiltonHeaterShakerBackend(index=2, interface=control_interface)

hs2 = HeaterShaker(
  name="Hamilton HeaterShaker",
  backend=backend2,
  size_x=146.2,
  size_y=103.8,
  size_z=74.11,
  child_location=Coordinate(x=9.66, y=9.22, z=74.11),
)

In [ ]:
await lh.setup()

for hs in [hs1, hs2]:
  await hs.setup()